In [ ]:
# Baixando
!pip install -U -q PyDrive

In [ ]:
# Importando bibliotecas
import pandas as pd
import numpy as np
from sklearn import svm
from pydrive.auth import GoogleAuth

from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google_drive_downloader import GoogleDriveDownloader as gdd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Autenticando para entrar no drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Baixando o arquivo de metadados
downloaded = drive.CreateFile({'id': '10MECbH6adkz4U4Y1arfhryv0-8zQas7w'})
downloaded.GetContentFile('data.csv')
print(downloaded['title'], downloaded['mimeType'])

data.csv text/csv


In [ ]:
# Lendo os dados do arquivo
df = pd.read_csv('data.csv')

In [ ]:
#Seleciona somente clientes
preframe = df.loc[df['num_produtos'] > 0]
preframe

,anomes,num_cpf_hash,vlr_credito,vlr_saldo,num_atend_atrs,vlr_score,num_produtos,num_atend,qtd_oper,qtd_reclm,qtd_restr,vlr_renda,cod_rating,ind_atrito,ind_engaj,ind_novo_cli
1,202104,ffffd54b45ec46113523184fc07185a0d5cbfa876a07ba...,35943.74,5815.50,NaN,377.0,1.0,NaN,24.0,NaN,10.0,NaN,A,NaN,1.0,NaN
4,202104,ffffbd4a3d42a12e07b1202d68c33d43220c42c8a55160...,1238.93,1400.00,NaN,773.0,2.0,NaN,21.0,NaN,NaN,NaN,A,NaN,1.0,NaN
5,202104,ffffafb315cf6852f08137c4486cc5d52df62c75da4aeb...,16368.03,NaN,NaN,391.0,1.0,NaN,6.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN
8,202104,ffff4ef886c28af8029c6b7d504942e6c9ef13021e9a35...,81506.54,5243.45,NaN,560.0,1.0,NaN,14.0,NaN,NaN,NaN,A,NaN,NaN,NaN
9,202104,ffff4bbb4074d7894174fd94c685edb3fd7fc9b7b1e3c1...,112918.62,7252.64,NaN,374.0,2.0,NaN,23.0,NaN,NaN,NaN,A,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12032485,202204,00008166c6d8213d5372a4a6336322f14e7f15b982cf5d...,43054.69,2620.89,NaN,437.0,2.0,NaN,17.0,NaN,8.0,NaN,A,NaN,NaN,NaN
12032487,202204,0000567d0d7f3813befdc85b65838d685ec6d89bb067bd...,21959.05,4017.00,NaN,410.0,1.0,NaN,15.0,NaN,4.0,NaN,H,NaN,NaN,NaN
12032488,202204,000055dd71d2394bf27d4fb46f4e8a91ba90a604774953...,970.18,NaN,NaN,415.0,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12032489,202204,00003a6cf049e4fe3416c1a55d1fc1aac613956ecd5d7a...,0.00,NaN,NaN,689.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Definindo o dataset com clientes e com o feature engineering selecionado
df_new_all = preframe[['anomes', 'vlr_score', 'vlr_saldo', 'num_atend', 'num_atend_atrs', 'qtd_oper', 'qtd_reclm', 'cod_rating' , 'ind_atrito', 'qtd_restr']].fillna(0)
df_new_focus = df_new_all[['vlr_score', 'vlr_saldo', 'num_atend', 'num_atend_atrs', 'qtd_oper', 'qtd_reclm', 'cod_rating' , 'ind_atrito', 'qtd_restr']]
df_new_focus

,vlr_score,vlr_saldo,num_atend,num_atend_atrs,qtd_oper,qtd_reclm,cod_rating,ind_atrito,qtd_restr
1,377.0,5815.50,0.0,0.0,24.0,0.0,A,0.0,10.0
4,773.0,1400.00,0.0,0.0,21.0,0.0,A,0.0,0.0
5,391.0,0.00,0.0,0.0,6.0,0.0,0,0.0,3.0
8,560.0,5243.45,0.0,0.0,14.0,0.0,A,0.0,0.0
9,374.0,7252.64,0.0,0.0,23.0,0.0,A,0.0,0.0
...,...,...,...,...,...,...,...,...,...
12032485,437.0,2620.89,0.0,0.0,17.0,0.0,A,0.0,8.0
12032487,410.0,4017.00,0.0,0.0,15.0,0.0,H,0.0,4.0
12032488,415.0,0.00,0.0,0.0,4.0,0.0,0,0.0,0.0
12032489,689.0,0.00,0.0,0.0,1.0,0.0,0,0.0,0.0


In [ ]:
#ordenando o dataset pelas colunas com IQR diferente de 0 (colunas relevantes)
df_util= df_new_focus.sort_values(by=['vlr_score','vlr_saldo','qtd_oper','qtd_restr'])
df_util

,vlr_score,vlr_saldo,num_atend,num_atend_atrs,qtd_oper,qtd_reclm,cod_rating,ind_atrito,qtd_restr
776,0.0,0.00,0.0,0.0,0.0,0.0,0,0.0,0.0
1136,0.0,0.00,0.0,0.0,0.0,0.0,0,0.0,0.0
1928,0.0,0.00,0.0,0.0,0.0,0.0,0,0.0,0.0
2392,0.0,0.00,0.0,0.0,0.0,0.0,0,0.0,0.0
2916,0.0,0.00,0.0,0.0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
11289499,1000.0,13810.00,0.0,0.0,19.0,0.0,A,0.0,0.0
10588464,1000.0,14649.52,0.0,0.0,38.0,0.0,A,0.0,0.0
7901562,1000.0,14649.61,0.0,0.0,45.0,0.0,A,0.0,0.0
7765198,1000.0,15050.40,0.0,0.0,10.0,0.0,A,0.0,0.0


In [ ]:
df_util.shape

(6218119, 9)

In [ ]:
x = 5/100
rejeicao = int(6218119 * x)
rejeicao

310905

In [ ]:
df_wth_out_outliers = df_util.iloc[ 310905 : -310905 ]
df_wth_out_outliers

,vlr_score,vlr_saldo,num_atend,num_atend_atrs,qtd_oper,qtd_reclm,cod_rating,ind_atrito,qtd_restr
7313912,0.0,19052.86,0.0,0.0,9.0,0.0,A,0.0,0.0
7275220,0.0,19053.03,0.0,0.0,24.0,0.0,B,0.0,0.0
7000755,0.0,19054.09,0.0,0.0,5.0,0.0,A,0.0,3.0
7400067,0.0,19055.00,0.0,0.0,19.0,0.0,A,0.0,0.0
6972028,0.0,19055.13,0.0,0.0,10.0,0.0,H,0.0,4.0
...,...,...,...,...,...,...,...,...,...
4773479,902.0,1360.00,0.0,0.0,9.0,0.0,A,0.0,0.0
8931872,902.0,1360.00,0.0,0.0,9.0,0.0,A,0.0,0.0
8405934,902.0,1360.00,0.0,0.0,11.0,0.0,A,0.0,0.0
5536165,902.0,1360.00,0.0,0.0,22.0,0.0,A,0.0,0.0


In [ ]:
df_atr = df_wth_out_outliers.loc[df['ind_atrito'] == 1]
x = df_atr
x

,vlr_score,vlr_saldo,num_atend,num_atend_atrs,qtd_oper,qtd_reclm,cod_rating,ind_atrito,qtd_restr
7470617,0.0,19809.69,0.0,0.0,31.0,1.0,A,1.0,0.0
10083927,0.0,19812.44,0.0,0.0,38.0,0.0,A,1.0,0.0
6712831,0.0,20198.45,0.0,0.0,35.0,0.0,A,1.0,1.0
7190966,0.0,21449.00,1.0,1.0,29.0,1.0,A,1.0,0.0
11182144,0.0,21905.94,0.0,0.0,11.0,1.0,A,1.0,1.0
...,...,...,...,...,...,...,...,...,...
1997940,900.0,5211.35,0.0,0.0,13.0,0.0,A,1.0,0.0
7869996,900.0,7017.00,0.0,0.0,8.0,0.0,A,1.0,0.0
8251022,900.0,23827.08,0.0,0.0,14.0,0.0,A,1.0,0.0
5792633,900.0,71145.96,0.0,0.0,52.0,0.0,A,1.0,0.0


In [ ]:
df_natr = df_wth_out_outliers.loc[df['ind_atrito'] != 1]
ys = df_natr.sample(frac=1)

In [ ]:
#Setando para 30/70
df_natr = df_wth_out_outliers.loc[df['ind_atrito'] != 1]
y1 = ys.sample(frac=1).head(6062)
y1

,vlr_score,vlr_saldo,num_atend,num_atend_atrs,qtd_oper,qtd_reclm,cod_rating,ind_atrito,qtd_restr
7935962,609.0,1752.96,0.0,0.0,7.0,0.0,A,0.0,0.0
1353999,575.0,13170.01,0.0,0.0,22.0,0.0,A,0.0,0.0
2552908,473.0,4918.56,0.0,0.0,31.0,0.0,A,0.0,0.0
4376175,122.0,0.00,0.0,0.0,0.0,0.0,0,0.0,2.0
2472731,475.0,1500.00,0.0,0.0,39.0,0.0,A,0.0,0.0
...,...,...,...,...,...,...,...,...,...
11098035,396.0,0.00,0.0,0.0,14.0,0.0,0,0.0,0.0
9892266,540.0,3985.12,0.0,0.0,3.0,0.0,A,0.0,0.0
11833271,318.0,142.78,0.0,0.0,1.0,0.0,G,0.0,1.0
3033150,743.0,4000.00,0.0,0.0,2.0,0.0,A,0.0,0.0


In [ ]:
#Setando para 40/60
df_natr = df_wth_out_outliers.loc[df['ind_atrito'] != 1]
y2 = ys.sample(frac=1).head(3897)
y2

,vlr_score,vlr_saldo,num_atend,num_atend_atrs,qtd_oper,qtd_reclm,cod_rating,ind_atrito,qtd_restr
7935962,609.0,1752.96,0.0,0.0,7.0,0.0,A,0.0,0.0
1353999,575.0,13170.01,0.0,0.0,22.0,0.0,A,0.0,0.0
2552908,473.0,4918.56,0.0,0.0,31.0,0.0,A,0.0,0.0
4376175,122.0,0.00,0.0,0.0,0.0,0.0,0,0.0,2.0
2472731,475.0,1500.00,0.0,0.0,39.0,0.0,A,0.0,0.0
...,...,...,...,...,...,...,...,...,...
7489853,0.0,20434.43,0.0,0.0,5.0,0.0,A,0.0,0.0
6056147,404.0,1788.77,0.0,0.0,13.0,0.0,A,0.0,0.0
11056372,369.0,635.78,0.0,0.0,9.0,0.0,AA,0.0,2.0
1267362,628.0,1832.05,0.0,0.0,7.0,0.0,A,0.0,0.0


In [ ]:
SafraArtificial1 = x.append(y1)
SafraArtificial1

,vlr_score,vlr_saldo,num_atend,num_atend_atrs,qtd_oper,qtd_reclm,cod_rating,ind_atrito,qtd_restr
7470617,0.0,19809.69,0.0,0.0,31.0,1.0,A,1.0,0.0
10083927,0.0,19812.44,0.0,0.0,38.0,0.0,A,1.0,0.0
6712831,0.0,20198.45,0.0,0.0,35.0,0.0,A,1.0,1.0
7190966,0.0,21449.00,1.0,1.0,29.0,1.0,A,1.0,0.0
11182144,0.0,21905.94,0.0,0.0,11.0,1.0,A,1.0,1.0
...,...,...,...,...,...,...,...,...,...
11098035,396.0,0.00,0.0,0.0,14.0,0.0,0,0.0,0.0
9892266,540.0,3985.12,0.0,0.0,3.0,0.0,A,0.0,0.0
11833271,318.0,142.78,0.0,0.0,1.0,0.0,G,0.0,1.0
3033150,743.0,4000.00,0.0,0.0,2.0,0.0,A,0.0,0.0


In [ ]:
SafraArtificial2 = x.append(y2)
SafraArtificial2

,vlr_score,vlr_saldo,num_atend,num_atend_atrs,qtd_oper,qtd_reclm,cod_rating,ind_atrito,qtd_restr
7470617,0.0,19809.69,0.0,0.0,31.0,1.0,A,1.0,0.0
10083927,0.0,19812.44,0.0,0.0,38.0,0.0,A,1.0,0.0
6712831,0.0,20198.45,0.0,0.0,35.0,0.0,A,1.0,1.0
7190966,0.0,21449.00,1.0,1.0,29.0,1.0,A,1.0,0.0
11182144,0.0,21905.94,0.0,0.0,11.0,1.0,A,1.0,1.0
...,...,...,...,...,...,...,...,...,...
7489853,0.0,20434.43,0.0,0.0,5.0,0.0,A,0.0,0.0
6056147,404.0,1788.77,0.0,0.0,13.0,0.0,A,0.0,0.0
11056372,369.0,635.78,0.0,0.0,9.0,0.0,AA,0.0,2.0
1267362,628.0,1832.05,0.0,0.0,7.0,0.0,A,0.0,0.0


In [ ]:
from google.colab import files

SafraArtificial1.to_csv('SafraArtificial1.csv', encoding = 'utf-8-sig') 
files.download('SafraArtificial1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
SafraArtificial2.to_csv('SafraArtificial2.csv', encoding = 'utf-8-sig') 
files.download('SafraArtificial2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>